# CHAPTER 4
## WQD7002 (P2) 
## WQD170031

## Steps thought of, will need to write them in for chapter 3 of the full report.
<br>1. data understanding - to identify fields important to the analysis (data accuracy etc.)
<br>2. text preprocessing - first remove common word (use nltk?); then perform 1 hot encoding (depending on how data need to be preprocessed to use the association rule library)???
<br>3. market basket analysis - association rule analysis - use pre-written libraries or copy from the coder who wrote his own algo for association rule analysis?
<br>4. evaluation by benchmarking to other metrics - see if high lift or support correspond to high number of lenders lending for the loan (or high amount raised in the crowdfunding) (or lesser time taken to raise the loan)

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#try see if the code above stays the same next session
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls '/content/drive/My Drive/kiva data/from build.kiva.org/'

In [0]:
my_folder = '/content/drive/My Drive/kiva data/from build.kiva.org/'

In [0]:
import pandas as pd
import numpy as np
import datetime

In [0]:
import pandas as pd
import numpy as np

df=pd.read_csv(my_folder+'loans_v2.csv')

df[['POSTED_TIME','PLANNED_EXPIRATION_TIME','DISBURSE_TIME','RAISED_TIME','REPORT_DT']] = \
df[['POSTED_TIME','PLANNED_EXPIRATION_TIME','DISBURSE_TIME','RAISED_TIME','REPORT_DT']].apply(pd.to_datetime)

df[df.select_dtypes(include=['int']).columns]=df[df.select_dtypes(include=['int']).columns].apply(pd.to_numeric,downcast='unsigned')

df[df.select_dtypes(include=['float']).columns]=df[df.select_dtypes(include=['float']).columns].apply(pd.to_numeric,downcast='float')

df[['ORIGINAL_LANGUAGE','STATUS','ACTIVITY_NAME','SECTOR_NAME','COUNTRY_CODE','COUNTRY_NAME','CURRENCY_POLICY','CURRENCY','REPAYMENT_INTERVAL','DISTRIBUTION_MODEL']]=\
df[['ORIGINAL_LANGUAGE','STATUS','ACTIVITY_NAME','SECTOR_NAME','COUNTRY_CODE','COUNTRY_NAME','CURRENCY_POLICY','CURRENCY','REPAYMENT_INTERVAL','DISTRIBUTION_MODEL']].astype('category')

In [0]:
df.info()

In [0]:
import dask.dataframe as dd
df=dd.read_csv(my_folder+'loans_v2.csv')

## Load dataset

In [0]:
#store the processes above as a python function
def data_overview(data):
  tmp=data.isnull().sum().reset_index(name='n_miss').rename({'index':'variable'},axis=1) #calculate number of missing values
  tmp2=data.notnull().sum().reset_index(name='n_notmiss').rename({'index':'variable'},axis=1) #calculate number of non-missing values, https://stackoverflow.com/questions/47044183/count-non-null-values-in-pandas
  tmp=pd.merge(tmp,tmp2,how='left',left_on='variable',right_on='variable') #join the tables so that can recycle variable name
  tmp['n']=tmp['n_miss']+tmp['n_notmiss']
  tmp2=data.nunique().reset_index(name='n_unique').rename({'index':'variable'},axis=1) #calculate number of unique values
  tmp=pd.merge(tmp,tmp2,how='left',left_on='variable',right_on='variable')
  tmp['miss_rate']=tmp['n_miss']/tmp['n']
  tmp['fill_rate']=tmp['n_notmiss']/tmp['n']
  tmp['distinct_rate']=tmp['n_unique']/tmp['n']
  tmp2=data.dtypes.reset_index(name='vartype').rename({'index':'variable'},axis=1) #get variable type
  tmp=pd.merge(tmp,tmp2,how='left',left_on='variable',right_on='variable')
  #tmp.to_excel(my_folder+'dataset_overview.xls')
  return tmp

In [0]:
df.head()

### Generate additional fields for analysis purposes

-might also be used in the results part depending on how i see the direction of the research report?
<br>-<DEL>borrower genders into number of males, number of females and number of borrowers</del>
<br><del>-borrower_pictured into number_of_borrowers_pictured (actually can drop this idea coz shown below same to number of borrowers)</del>
<br>-number of sentences, <del>number of words</del> in description and description_translated
-[don't feel like the number of sentences is achievable, coz dots were also used for name short forms.]
<br><del>-combine description to description_translated for english to increase fill rate, call description_new.
<br>-number of words in loan use
<br>-number of words in  activity name</del>


In [0]:
#check number of rows in the dataset
df['DESC_TRANSLATED_WORDCOUNT'] = df['DESCRIPTION_TRANSLATED'].str.split().str.len() #https://stackoverflow.com/questions/37483470/how-to-calculate-number-of-words-in-a-string-in-dataframe
#df['DESC_TRANSLATED_SENTENCE_COUNT'] = df['DESCRIPTION_TRANSLATED'].str.split().str.len()
#derive variable counting number of description to identify duplicates
df['DESC_WORDCOUNT'] = df['DESCRIPTION'].str.split().str.len()
#df['DESC_SENTENCE_COUNT'] = df['DESCRIPTION'].str.split().str.len()
#derive variable counting number of description to identify duplicates

df['DESCRIPTION_NEW'] = np.where(df['ORIGINAL_LANGUAGE']=='English', df['DESCRIPTION'], df['DESCRIPTION_TRANSLATED'])   #combine description and description_translated
df['DESC_NEW_WORDCOUNT'] = df['DESCRIPTION_NEW'].str.split().str.len()
#df['DESC_NEW_SENTENCE_COUNT'] = df['DESCRIPTION_NEW'].str.split().str.len()
#derive variable to count number of description to identify duplicates

df['LOAN_USE_WORDCOUNT'] = df['LOAN_USE'].str.split().str.len()
#derive variable to check number of sentences in loan use
df['AVY_NAME_WORDCOUNT'] = df['ACTIVITY_NAME'].str.split().str.len()
#derive variable to check number of sentences in activity name

#variable to get number of females and males (borrowers)

#df['DESC_WORDCOUNT'].value_counts(ascending=True).to_csv(my_folder+'desc_wordcount.csv')
#df['DESC_WORDCOUNT'].value_counts(ascending=True).reset_index(name='freq').freq.value_counts().\
#reset_index().freq.describe(percentiles=[.01,.05,.1,.25,.5,.75,.9,.95,.99])#.to_csv(my_folder+'wordcount_freq.csv')

In [0]:
df['AVY_NAME_CLEAN_LM_WORDCOUNT'] = df['AVY_NAME_CLEAN_LEMMA'].str.split().str.len()
df['AVY_NAME_CLEAN_PS_WORDCOUNT'] = df['AVY_NAME_CLEAN_PS'].str.split().str.len()

df['TAGS_CLEAN_LM_WORDCOUNT'] = df['TAGS_CLEAN_LEMMA'].str.split().str.len()
df['TAGS_CLEAN_PS_WORDCOUNT'] = df['TAGS_CLEAN_PS'].str.split().str.len()

df['LOAN_USE_CLEAN_LM_WORDCOUNT'] = df['LOAN_USE_CLEAN_LEMMA'].str.split().str.len()
df['LOAN_USE_CLEAN_PS_WORDCOUNT'] = df['LOAN_USE_CLEAN_PS'].str.split().str.len()

In [0]:
#count number of repeated description
#https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html
tmp=df.groupby('DESCRIPTION_TRANSLATED').size().reset_index(name='N_DESC_TRANSLATED')
df=pd.merge(df,tmp,how='left',left_on='DESCRIPTION_TRANSLATED',right_on='DESCRIPTION_TRANSLATED')

tmp=df.groupby('DESCRIPTION').size().reset_index(name='N_DESC')
df=pd.merge(df,tmp,how='left',left_on='DESCRIPTION',right_on='DESCRIPTION')

tmp=df.groupby('DESCRIPTION_NEW').size().reset_index(name='N_DESC_NEW')
df=pd.merge(df,tmp,how='left',left_on='DESCRIPTION_NEW',right_on='DESCRIPTION_NEW')

In [0]:
df['N_BORROWERS'] = df['BORROWER_GENDERS'].str.split().str.len()
df['N_BORROWERS2'] = df['BORROWER_PICTURED'].str.split().str.len() #created as cross-check against variable built above
df['GROUP_LOANS']=np.where(df['N_BORROWERS']>1,"group","individual")

df['N_TAGS'] = df['TAGS'].str.split().str.len()

df['N_F_BORROWERS']=df['BORROWER_GENDERS'].str.lower().str.count("female")
df['N_M_BORROWERS']=df['N_BORROWERS']-df['N_F_BORROWERS']

df['HAVE_PICTURE']=np.where(df['IMAGE_ID'].isnull(),0,1)
df['HAVE_VIDEO']=np.where(df['VIDEO_ID'].isnull(),0,1)
df['N_BORROWERS_PICTURED']=df['BORROWER_PICTURED'].str.lower().str.count("true")  #n borrowers pictured
#df['BORROWER_GENDERS'].head(5)

In [0]:
df['N_TAGS'] = df['TAGS'].str.split().str.len()

In [0]:
#to use as a way to countercheck success in fundraising activity on Kiva.org
#if -ve then unsuccessful and we should expect this to tally with STATUS='REFUNDED/EXPIRED'
#if value >=0 means it is successful
#at the same time can inspect of the value can exceed 0:
##would like to expect that the value will never exceed 0
##once the requested amount have been fulfilled then the funding for that particular loan closes
df['D_RAISED_LOANED']=df['FUNDED_AMOUNT']-df['LOAN_AMOUNT']

### generate additional time related fields
-understanding the time frame in the data downloaded
<br>-suspect that RAISED_TIME is time when a loan is fully funded by Kiva lenders on Kiva
<br>--if this is true, then the dates should always be later than POSTED_TIME, which is time when a loan is posted by FP for fundraising in Kiva.
<br><del>-compare POSTED_TIME and DISBURSED_TIME as a way to check which loans have been disbursed first from FP to borrower</del> coded 
<br>-suspect that PLANNED_EXPIRATION_TIME is the time allocated/targeted to raise the fund
<br>--if the above is true then this value should be really small (not more than 3 months)
<br><del>-compare PLANNED_EXPIRATION_TIME and DISBURSED_TIME, and PLANNED_EXPIRATION_TIME and POSTED_TIME</del> coded 
<br>--to verify whether PLANNED_EXPIRATION_TIME is same as LENDER_TERM or different
<BR>--if above is true then then LENDER_TERM should have a value closely associated with one of the calculated fields


In [0]:
#library to be used
#http://www.rohitschauhan.com/index.php/2018/06/13/python-date-manipulation-for-sas-users/
import datetime as dt
import numpy as np
import re
import string

In [0]:
#calculate the time fields
#end_date_series.dt.to_period('M') - start_date_series.dt.to_period('M') 

#suspect that RAISED_TIME is time when a loan is fully funded by Kiva lenders on Kiva
##if this is true, then the dates should always be later than POSTED_TIME,
##which is time when a loan is posted by FP for fundraising in Kiva.
df['N_MTH_POSTED_RAISED']=df['RAISED_TIME'].dt.to_period('M') - df['POSTED_TIME'].dt.to_period('M')

#compare POSTED_TIME and DISBURSED_TIME as a way to check which loans have been disbursed first from FP to borrower 
df['N_MTH_DISBURSED_POSTED']=df['POSTED_TIME'].dt.to_period('M')-df['DISBURSE_TIME'].dt.to_period('M') #should have more +ve values here

#compare PLANNED_EXPIRATION_TIME and DISBURSED_TIME, and PLANNED_EXPIRATION_TIME and POSTED_TIME 
##to verify whether PLANNED_EXPIRATION_TIME is same as LENDER_TERM or different
df['N_MTH_DISBURSED_EXPIRE']=df['PLANNED_EXPIRATION_TIME'].dt.to_period('M')-df['DISBURSE_TIME'].dt.to_period('M')
df['N_MTH_POSTED_EXPIRE']=df['PLANNED_EXPIRATION_TIME'].dt.to_period('M')-df['POSTED_TIME'].dt.to_period('M')

/usr/local/lib/python3.6/dist-packages/pandas/core/arrays/datetimes.py:1269: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [0]:
#https://stackoverflow.com/questions/43876281/extract-all-numeric-characters-from-a-pandas-series-all-groups
#x['clean'] = x.phone.str.replace(r'\D+', '')

#df['N_MTH_POSTED_RAISED']=df['N_MTH_POSTED_RAISED'].str.replace(r'\D+','')
#df['N_MTH_DISBURSED_POSTED']=df['N_MTH_DISBURSED_POSTED'].str.replace(r'\D+','')
#df['N_MTH_DISBURSED_EXPIRE']=df['N_MTH_DISBURSED_EXPIRE'].str.replace(r'\D+','')
#df['N_MTH_POSTED_EXPIRE']=df['N_MTH_POSTED_EXPIRE'].str.replace(r'\D+','')

df[['N_MTH_POSTED_RAISED','N_MTH_DISBURSED_POSTED','N_MTH_DISBURSED_EXPIRE','N_MTH_POSTED_EXPIRE']]=\
df[['N_MTH_POSTED_RAISED','N_MTH_DISBURSED_POSTED','N_MTH_DISBURSED_EXPIRE','N_MTH_POSTED_EXPIRE']].apply(pd.to_numeric)

df[['N_MTH_POSTED_RAISED','N_MTH_DISBURSED_POSTED','N_MTH_DISBURSED_EXPIRE','N_MTH_POSTED_EXPIRE']].head(10)
df[['N_MTH_POSTED_RAISED','N_MTH_DISBURSED_POSTED','N_MTH_DISBURSED_EXPIRE','N_MTH_POSTED_EXPIRE']].dtypes

In [0]:
df['REPORT_DT'] = df['POSTED_TIME'] + pd.offsets.MonthEnd()
df[['REPORT_DT']].head()

In [0]:
#checkpoint
df.to_csv(my_folder+'loans_v2.csv',index=False)

In [0]:
#https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sas.html
#split the date information by year, month  
#try to consider time of day, week of month, week of year, day (Sun-Fri)
#do this based on posted_time
df['POSTED_YEAR'] = df['POSTED_TIME'].dt.year
df[['POSTED_YEAR']].head()

In [0]:
#https://stackoverflow.com/questions/51287504/remove-the-time-from-datetime-datetime-in-pandas-column
#http://www.datasciencemadesimple.com/difference-two-timestamps-seconds-minutes-hours-pandas-python-2/
#from datetime import date

#d0 = date(2008, 8, 18)
#d1 = date(2008, 9, 26)
#delta = d1 - d0
#print(delta.days)

df['N_DAYS_POSTED_RAISED']=df['RAISED_TIME'].dt.date - df['POSTED_TIME'].dt.date

#compare POSTED_TIME and DISBURSED_TIME as a way to check which loans have been disbursed first from FP to borrower 
#df['N_DAYS_DISBURSED_POSTED']=df['POSTED_TIME'].dt.date - df['DISBURSE_TIME'].dt.date #should have more +ve values here
df['N_HRS_POSTED_RAISED']=((df['RAISED_TIME']-df['POSTED_TIME'])/np.timedelta64(1,'h')).round()

df[['N_HRS_POSTED_RAISED','N_DAYS_POSTED_RAISED']].head()

,N_HRS_POSTED_RAISED,N_DAYS_POSTED_RAISED
0,13.0,0 days
1,215.0,9 days
2,25.0,1 days
3,15.0,0 days
4,118.0,5 days


In [0]:
#https://stackoverflow.com/questions/46786211/counting-the-frequency-of-words-in-a-pandas-data-frame
#df.ACTIVITY_NAME.str.split(expand=True).stack().value_counts().to_excel(my_folder+'avy_name_wordfreq.xls')
#df.LOAN_USE.str.split(expand=True).stack().value_counts().to_excel(my_folder+'loan_use_wordfreq.xlsx')
df.DESCRIPTION.str.split(expand=True).stack().value_counts().to_excel(my_folder+'desc_wordfreq.xlsx')
df.DESCRIPTION_NEW.str.split(expand=True).stack().value_counts().to_excel(my_folder+'desc_new_wordfreq.xlsx')
df.DESCRIPTION_TRANSLATED.str.split(expand=True).stack().value_counts().to_excel(my_folder+'desc_trans_wordfreq.xlsx')

TypeError: ignored

In [0]:
#df['DESC_WORDCOUNT'].value_counts(ascending=True).to_csv(my_folder+'desc_wordcount.csv')
df['DESC_WORDCOUNT'].value_counts(ascending=True).reset_index(name='freq').freq.value_counts().\
#reset_index().freq.describe(percentiles=[.01,.05,.1,.25,.5,.75,.9,.95,.99])#.to_csv(my_folder+'wordcount_freq.csv')

## Charts to understand the datasets

### functions defined for plotting charts

In [0]:
#create a function to generate the plotly horizontal bar chart for frequencies of categorical variables
#https://stackoverflow.com/questions/35692781/python-plotting-percentage-in-seaborn-bar-plot
 
import plotly.graph_objects as go
import seaborn as sns
import plotly.express as px

def interactive_freqchart_h(data,variable):
   
   #generate frequency dataset
   tmp=data[variable].value_counts(normalize=True).\
   reset_index(name='freq').rename(mapper={'index':variable},axis=1)#.\
   #pipe((sns.barplot, "data"), x='freq', y='ORIGINAL_LANGUAGE', palette='ch:.25')
   
   tmp['freq']=tmp['freq']*100
   #sns.barplot(y='ORIGINAL_LANGUAGE',x='freq',data=tmp,order=tmp['ORIGINAL_LANGUAGE'])

   fig = go.Figure(go.Bar(
       x=tmp['freq'],
       y=tmp[variable],
       orientation='h'))
   fig.update_layout(
       title="Distribution of " + variable,
       xaxis_title="Frequency (in %)",
       yaxis_title=variable)
   fig.show()
   #fig.show(config={'editable': True,'scrollZoom': True,'displayModeBar': True})
   #'modeBarButtonsToRemove': ['toggleSpikelines','hoverCompareCartesian']

In [0]:
import plotly.graph_objects as go

def interactive_freqchart_v(data,variable):
   
   #generate frequency dataset
   tmp=data[variable].value_counts(normalize=True).\
   reset_index(name='freq').rename(mapper={'index':variable},axis=1)
   
   tmp['freq']=tmp['freq']*100

   fig = go.Figure(go.Bar(
       x=tmp[variable],
       y=tmp['freq']))
   fig.update_layout(
       title="Distribution of " + variable,
       xaxis_title=variable,
       yaxis_title="Frequency (in %)")
   fig.show()
   #fig.show(config={'editable': True,'scrollZoom': True,'displayModeBar': True})
   #'modeBarButtonsToRemove': ['toggleSpikelines','hoverCompareCartesian']

In [0]:
#plan to use boxplots later, https://plot.ly/python/box-plots/
import plotly.express as px
tips = px.data.tips()
fig = px.box(tips, x="time", y="total_bill")
fig.show()

## Step 2: text preprocessing
(Step 1 data understanding is covered inside another ipynb)
<br>It was very time consuming to pre-process the description field. try to process based on loan_use or activity_name first.

In [0]:
#df=pd.read_csv('E:/New folder/Google Drive/isha.salleh - backup and sync/kiva data/from build.kiva.org/loans_v2.csv')
df=pd.read_csv('/content/drive/My Drive/kiva data/from build.kiva.org/loans_v3.csv')

In [0]:
df['DESC_WORDCOUNT'].describe(percentiles=[.01,.05,.1,.25,.5,.75,.9,.95,.99])
#consider adding a chart to visualize the calculated DESC_WORDCOUNT

count    1.229155e+06
mean     1.246997e+02
std      5.355588e+01
min      1.000000e+00
1%       4.900000e+01
5%       6.400000e+01
10%      7.100000e+01
25%      8.700000e+01
50%      1.120000e+02
75%      1.510000e+02
90%      1.950000e+02
95%      2.260000e+02
99%      2.990000e+02
max      1.521000e+03
Name: DESC_WORDCOUNT, dtype: float64

Word count analysis. 

<br>Have max 1521 words used in a single description of a loan. 
<br>May want to consider charting the frequency of DESC_WORDCOUNT to show how it looks like.
<br>(later add in observation from wordcount_freq.csv)
<br>most word counts seem to have value of 1

In [0]:
#extract first 5 rows to test whether algorithm works or not. 
df2=df[~df.DESCRIPTION_TRANSLATED.isnull()].head(5).copy() #https://stackoverflow.com/questions/39475566/python-pandas-filter-out-records-with-null-or-empty-string-for-a-given-field
df2.shape

(10, 61)

In [0]:
#URL 1: https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
#URL 2: https://www.geeksforgeeks.org/python-stemming-words-with-nltk/
#URL 3: https://medium.com/@datamonsters/text-preprocessing-in-python-steps-tools-and-examples-bf025f872908
#URL 4: https://blog.ekbana.com/pre-processing-text-in-python-ad13ea544dae
import re       #removal of special symbols (Manura's code) and numbers
import string   #used in URL3 for removing punctuation symbols
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem.porter import PorterStemmer  #what is porterstemmer used for - URL 2 seem to indicate stemmer used to identify words like "playing" etc.
from nltk.tokenize import word_tokenize     #from URL #1 looks like this converts each words separated by space, and symbols into individual terms. 
from nltk.stem import WordNetLemmatizer 
#from nltk.tokenize import LineTokenizer
#may be can just use this instead of using one-hot encoding

In [6]:
#had a problem with the stopwords just now....
nltk.download('stopwords')
nltk.download('punkt')  #to use word_tokenize
nltk.download('wordnet')  #to use WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
#https://www.nltk.org/book/
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
tag_map['N'] = wn.NOUN

In [0]:
tag_map

defaultdict(<function __main__.<lambda>>,
            {'J': 'a', 'N': 'n', 'R': 'r', 'V': 'v'})

In [0]:
#viewing stopwords 
len(stopwords.words('english'))

179

In [0]:
#https://www.geeksforgeeks.org/python-nltk-nltk-tokenize-conditionalfreqdist/
#try to have a better understanding of words to be removed by generating the frequency distribution of the terms
#apart from stemmer may want to consider using lemmatizer: https://www.geeksforgeeks.org/python-lemmatization-with-nltk/

In [0]:
#removal of unnecessary characters from the field
#may need to create additional corpus of common words based on borrower name to remove their name from the description column.
#add removal of symbols (comma, quote symbols from the sentence)
whos

Variable                  Type             Data/Info
----------------------------------------------------
InteractiveShell          MetaHasTraits    <class 'IPython.core.inte<...>eshell.InteractiveShell'>
d0                        date             2008-08-18
d1                        date             2008-09-26
date                      type             <class 'datetime.date'>
delta                     timedelta        39 days, 0:00:00
df                        DataFrame                 LOAN_ID  ... N_H<...>682790 rows x 74 columns]
drive                     module           <module 'google.colab.dri<...>s/google/colab/drive.py'>
dt                        module           <module 'datetime' from '<...>b/python3.6/datetime.py'>
go                        module           <module 'plotly.graph_obj<...>plotly/graph_objects.py'>
interactive_freqchart_h   function         <function interactive_fre<...>hart_h at 0x7f35740fd840>
interactive_freqchart_v   function         <function interactive_

### text cleaning function

In [0]:
#things to do to remove stopwords - too slow, what can be done to make it faster
#other concents that may not have been incorporated:
##1.URL 4 - contraction - expand words like "ain't" to "am not"
##2.URL 4 - spell check
##3.URL 2,3 & 4 - lemmatization and stemming

#https://thispointer.com/pandas-6-different-ways-to-iterate-over-rows-in-a-dataframe-update-while-iterating-row-by-row/
import datetime

def text_cleaning_lemma(data,varin,varout):
  desc_clean=[] #create a list to store cleaned up description field
  start_time=datetime.datetime.now()
  
  for row in data.itertuples():
    dictRow = row._asdict()
    if pd.isnull(dictRow[varin]):
      desc_clean.append(' ')
    else:
  #if description is nan then return empty space to desc_clean, otherwise proceed
      tmp=dictRow[varin].lower()  #1. extract the desription translated into separate words stored in a list
      tmp=tmp.strip('\\r\\n')   #https://stackoverflow.com/questions/45383938/cant-delete-r-n-from-a-string/45383951
      tmp=tmp.translate(str.maketrans("", "",string.punctuation))  #2. remove symbols from URL 3, improved with https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
      tmp = re.sub('rnrn', ' ', tmp) #some '\\r\\n' cant be removed earlier, so use https://stackoverflow.com/questions/3939361/remove-specific-characters-from-a-string-in-python
      tmp=re.sub(r'\d+','',tmp)   #2a. remove numbers (check whether necessary or not)
      tmp=re.sub('<[^<]+?>','',tmp)  #2c. remove HTML tags, based on URL 4
    #3. check whether the terms inside are common words or not, if not, store in a variable separated by space/commas that will be appended back to the df?
    #want to change sequence: start with lemmatization, followed by stemmer, then autocorrect.
      lemmatizer=WordNetLemmatizer()
      tmp=pos_tag(word_tokenize(tmp))
      #tmp=[lemmatizer.lemmatize(word,pos='v') for word in tmp if not word in set(stopwords.words('english'))]
      tmp=[lemmatizer.lemmatize(word,pos=tag_map[tag[0]]) for word,tag in tmp if not word in set(stopwords.words('english'))]
      tmp=' '.join(tmp)
      #print(tmp)
      desc_clean.append(tmp)  #4. store in a list

  data[varout]=desc_clean #5. append the list to df as a new df column
  #print('done in', datetime.datetime.now()-start_time,"!")
  return data

In [0]:
def text_cleaning_ps(data,varin,varout):
  start_time=datetime.datetime.now()
  desc_clean=[] #create a list to store cleaned up description field

  for row in data.itertuples():
    dictRow = row._asdict()
    if pd.isnull(dictRow[varin]):
      desc_clean.append(' ')
    else:
  #if description is nan then return empty space to desc_clean, otherwise proceed
      tmp=dictRow[varin].lower()  #1. extract the desription translated into separate words stored in a list
      tmp=tmp.strip('\\r\\n')   #https://stackoverflow.com/questions/45383938/cant-delete-r-n-from-a-string/45383951
      tmp=tmp.translate(str.maketrans("", "",string.punctuation))  #2. remove symbols from URL 3, improved with https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
      tmp = re.sub('rnrn', ' ', tmp) #some '\\r\\n' cant be removed earlier, so use https://stackoverflow.com/questions/3939361/remove-specific-characters-from-a-string-in-python
      tmp=re.sub(r'\d+','',tmp)   #2a. remove numbers (check whether necessary or not)
      tmp=re.sub('<[^<]+?>','',tmp)  #2c. remove HTML tags, based on URL 4
    #3. check whether the terms inside are common words or not, if not, store in a variable separated by space/commas that will be appended back to the df?
    #want to change sequence: start with lemmatization, followed by stemmer, then autocorrect.
     #try to skip stemming first?
      tmp=word_tokenize(tmp)
      ps=PorterStemmer()
      tmp=[ps.stem(word) for word in tmp if not word in set(stopwords.words('english'))]
      #3a. create own list of common words to further clean the field - names, geographical locations?
      tmp=' '.join(tmp)
      #print(tmp)
      desc_clean.append(tmp)  #4. store in a list

  data[varout]=desc_clean #5. append the list to df as a new df column
  #print('done in', datetime.datetime.now()-start_time,"!")
  return data

In [0]:
def custom_stopword_code(data,wordvar,freqvar,n=1):
  custom_stopword=[words for words in data[data[freqvar]<=n][wordvar]]
  #[words for words in data[wordvar] if data[freqvar]<=n]
  return custom_stopword

In [0]:
def text_cleaning_custom(data,varin,varout,vareval,stopword_data,freqvar,wordvar, n=1):
  start_time=datetime.datetime.now()
  desc_clean=[] #create a list to store cleaned up description field
  custom_stopword=[words for words in stopword_data[stopword_data[freqvar]<=n][wordvar]]
  custom_stopword.extend(['br','year','old','also','like','kiva','nwtf']) #only used for DESC_NEW field cleaning
  for row in data.itertuples():
    dictRow = row._asdict()
    if pd.isnull(dictRow[varin]):
      desc_clean.append(' ')
    else:
  #if description is nan then return empty space to desc_clean, otherwise proceed
      #tmp=word_tokenize(dictRow[varin].str)
      tmp=dictRow[varin].split()
      tmp=[word for word in tmp if not word in custom_stopword]
      tmp=[re.sub(r'\d+','',word) for word in tmp]
      tmp=[re.sub('<[^<]+?>','',word) for word in tmp]
      tmp=[word.translate(str.maketrans("", "",string.punctuation)) for word in tmp]
      #3a. create own list of common words to further clean the field - names, geographical locations?
      tmp=' '.join(tmp)
      #print(tmp)
      desc_clean.append(tmp)  #4. store in a list

  data[varout]=desc_clean #5. append the list to df as a new df column
  time_taken = datetime.datetime.now() - start_time
  return data ,time_taken

### start cleanup

In [0]:
#full swing clean up, clean up DESCRIPTION_NEW field only first

#df=text_cleaning_lemma(data=df,varin='LOAN_USE',varout='LOAN_USE_CLEAN_LEMMA')
#df=text_cleaning_lemma(data=df,varin='LOAN_USE',varout='LOAN_USE_CLEAN_PS')
#df=text_cleaning_ps(data=df,varin='ACTIVITY_NAME',varout='AVY_NAME_CLEAN_LEMMA')
#df=text_cleaning_ps(data=df,varin='ACTIVITY_NAME',varout='AVY_NAME_CLEAN_PS')
#df.to_csv(my_folder+'loans_v2.csv')

#df_year=df[(df['POSTED_YEAR']==2018)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_2018.csv', index=False)

#df_year=df[(df['POSTED_YEAR']==2017)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_2017.csv', index=False)

#df_year=df[(df['POSTED_YEAR']==2016)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_2016.csv', index=False)

#df_year=df[(df['POSTED_YEAR']==2015)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_2015.csv', index=False)

df_year=df[(df['POSTED_YEAR']==2014)].copy()
df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
df_year.to_csv(my_folder+'loans_2014.csv', index=False)

#df_year=df[(df['POSTED_YEAR']<=2008) | (df['POSTED_YEAR']>=2019)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_othyr.csv', index=False)

#df_year=df[(df['POSTED_YEAR']==2009)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_2009.csv', index=False)

#df_year=df[(df['POSTED_YEAR']==2010)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_2010.csv', index=False)

#df_year=df[(df['POSTED_YEAR']==2011)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_2011.csv', index=False)

#df_year=df[(df['POSTED_YEAR']==2012)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_2012.csv', index=False)

#df_year=df[(df['POSTED_YEAR']==2013)].copy()
#df_year=text_cleaning_lemma(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_LEMMA')
#df_year=text_cleaning_ps(data=df_year,varin='DESCRIPTION_NEW',varout='DESC_NEW_CLEAN_PS')
#df_year.to_csv(my_folder+'loans_2013.csv', index=False)

### further text cleaning, reduce words with very low frequency based on the wordfreq table generated.
try with the cleaned up loan use field and reduce bit by bit to see 

#### loan use field clean up

In [0]:
df2017=pd.read_csv(my_folder+'loans_2017_v2.csv')
#df2018=pd.read_csv(my_folder+'loans_2018.csv')
#loanuse_wordfreq=pd.read_excel(my_folder+'loan_use_wordfreq.xlsx')
df2017.columns

In [0]:
#filter out missing tags, description_new, loan_use, activity_name and fundraising cases.
#df2018.shape
#df2018 = df2018.dropna(axis=0, how='any', subset=['DESCRIPTION_NEW','LOAN_USE','ACTIVITY_NAME','TAGS'])
#df2018.shape 

#drop status fundraising
df2018['STATUS'].value_counts()
df2018 = df2018[~(df2018['STATUS']=="fundRaising")]
df2018['STATUS'].value_counts()

(229631, 77)

(187922, 77)

In [0]:
#filter out missing tags, description_new, loan_use, activity_name and fundraising cases.
df2017.shape
df2017 = df2017.dropna(axis=0, how='any', subset=['DESCRIPTION_NEW','LOAN_USE','ACTIVITY_NAME','TAGS'])
df2017.shape 

#drop status fundraising
df2017['STATUS'].value_counts()
df2017 = df2017[~(df2017['STATUS']=="fundRaising")]
df2017['STATUS'].value_counts()

(225476, 79)

(196075, 79)

funded     186998
expired      9077
Name: STATUS, dtype: int64

funded     186998
expired      9077
Name: STATUS, dtype: int64

In [0]:
loanuse_wordfreq=loanuse_wordfreq.rename(columns={'Unnamed: 0':'word',0:'freq'})
loanuse_wordfreq.columns

NameError: ignored

In [0]:
#data ,time_taken = text_cleaning_custom(data,varin,varout,vareval,stopword_data,freqvar,wordvar, n=1)

df2018, running_time = \
text_cleaning_custom(data = df2018,varin = 'LOAN_USE_CLEAN_LEMMA',varout = 'LOAN_USE_LM2',vareval = 'LOAN_USE',\
                     stopword_data = loanuse_wordfreq,freqvar = 'freq',wordvar = 'word', n=1)
print(running_time)
df2018['LOAN_USE_LM2'].head()

0:11:49.816937


0        tto pay tuition fee child report school month
1    tto buy farm input seed quality fertilizer hel...
2                           start build house land buy
3                         build sanitary toilet family
4                                             pay land
Name: LOAN_USE_LM2, dtype: object

In [0]:
df2018.columns
df2018=df2018.rename({'LOAN_USE_LM3':'LOAN_USE_LM3_OLD'}, axis=1)
df2018.columns

In [0]:
#df2018=df2018.drop(columns=['LOAN_USE_LM3'], axis=1)  #drop LM3 if still cant establish crosstab
#df2018.columns
df2018, running_time = \
text_cleaning_custom(data = df2018,varin = 'LOAN_USE_LM2',varout = 'LOAN_USE_LM3_V2',vareval = 'LOAN_USE',\
                     stopword_data = new_cleaning_2018,freqvar = 'freq',wordvar = 'loanuse_term', n=15)
print(running_time)
df2018['LOAN_USE_LM3_V2'].head()

0:05:03.435347


0        tto pay tuition fee child report school month
1    tto buy farm input seed quality fertilizer hel...
2                           start build house land buy
3                         build sanitary toilet family
4                                             pay land
Name: LOAN_USE_LM3_V2, dtype: object

In [0]:
df2018, running_time = \
text_cleaning_custom(data = df2018,varin = 'LOAN_USE_LM3_OLD',varout = 'LOAN_USE_LM3',vareval = 'LOAN_USE',\
                     stopword_data = new_cleaning_2018,freqvar = 'freq',wordvar = 'loanuse_term', n=15)
print(running_time)
df2018['LOAN_USE_LM3'].head()

0:04:52.431415


0               tto pay tuition fee child school month
1    tto buy farm input seed quality fertilizer hel...
2                           start build house land buy
3                         build sanitary toilet family
4                                             pay land
Name: LOAN_USE_LM3, dtype: object

#### desc_new cleanup


In [0]:
df2017=pd.read_csv(my_folder+'loans_2017_v3.csv')
df2018=pd.read_csv(my_folder+'loans_2018_v3.csv')

In [0]:
df2017['DESC_NEW_CLEAN_LEMMA'].str.split(expand=True).stack().value_counts().\
reset_index(name='freq').rename(columns={'index':'desc_new_term'}).\
to_excel(my_folder+'2017_desc_new_lm_wordfreq.xlsx')

df2018['DESC_NEW_CLEAN_LEMMA'].str.split(expand=True).stack().value_counts().\
reset_index(name='freq').rename(columns={'index':'desc_new_term'}).\
to_excel(my_folder+'2018_desc_new_lm_wordfreq.xlsx')

In [0]:
new_cleaning=pd.read_excel(my_folder+'2017_desc_new_lm_wordfreq.xlsx')
new_cleaning_2018=pd.read_excel(my_folder+'2018_desc_new_lm_wordfreq.xlsx')

In [0]:
#try to see what happens if try to clean up further 
#may need to ditch this and propose as4/tgFH1VihPo8xNxve-0YANth0_Rstgs_4-Db9lNLnmWBiIBlicFQHGh0 refinement in future work
#remove desc_new_term with wordfreq<300
#limit to <3000 columns, which is still cumbersome to eliminate association rules afterwards 
df2017, running_time = \
text_cleaning_custom(data = df2017,varin = 'DESC_NEW_CLEAN_LEMMA',varout = 'DESC_NEW_LM2',vareval = 'DESCRIPTION_NEW',\
                     stopword_data = new_cleaning,freqvar = 'freq',wordvar = 'desc_new_term', n=300)
print(running_time)

df2017['DESC_NEW_LM2'].str.split(expand=True).stack().value_counts().\
reset_index(name='freq').rename(columns={'index':'desc_new_term'}).shape

df2017['DESC_NEW_LM2'].head()

df2017.to_csv(my_folder+'loans_2017_v4.csv', index=False)

In [0]:
#may need to perform extra cleaning up to also remove terms used in 2017?
#if based on 2018_desc_new_wordfreq should elimiate wordfreq <=500
df2018, running_time = \
text_cleaning_custom(data = df2018,varin = 'DESC_NEW_CLEAN_LEMMA',varout = 'DESC_NEW_TMP',vareval = 'DESCRIPTION_NEW',\
                     stopword_data = new_cleaning,freqvar = 'freq',wordvar = 'desc_new_term', n=300)
print(running_time)

df2018['DESC_NEW_TMP'].str.split(expand=True).stack().value_counts().\
reset_index(name='freq').rename(columns={'index':'desc_new_term'}).shape

df2018, running_time = \
text_cleaning_custom(data = df2018,varin = 'DESC_NEW_TMP',varout = 'DESC_NEW_LM2',vareval = 'DESCRIPTION_NEW',\
                     stopword_data = new_cleaning_2018,freqvar = 'freq',wordvar = 'desc_new_term', n=300)
print(running_time)

df2018['DESC_NEW_LM2'].str.split(expand=True).stack().value_counts().\
reset_index(name='freq').rename(columns={'index':'desc_new_term'}).shape

df2018['DESC_NEW_TMP'].head()

df2018.to_csv(my_folder+'loans_2018_v4.csv', index=False)

### check frequency of terms to check for further cleaning that will be needed

In [0]:
loanuse_wordfreq.shape

(100338, 2)

In [0]:
new_cleaning=pd.read_excel(my_folder+'loan_use_lm2_wordfreq.xlsx')

In [0]:
#data[variable]
new_cleaning=df2017['LOAN_USE_LM2'].str.split(expand=True).stack().value_counts().\
reset_index(name='Freq').rename(columns={'index':'loanuse_term'})#.shape
#\.to_excel(my_folder+'desc_wordfreq.xlsx')
new_cleaning['percent']=new_cleaning['Freq']/new_cleaning['Freq'].sum()
new_cleaning.head(5)
new_cleaning.shape
#new_cleaning.to_excel(my_folder+'loan_use_lm2_wordfreq.xlsx')

In [0]:
df2017['LOAN_USE_LM3'].str.split(expand=True).stack().value_counts().\
reset_index(name='freq').rename(columns={'index':'loanuse_term'}).shape

(2694, 2)

In [0]:
#new_cleaning_2018=
df2018['LOAN_USE_LM3_OLD'].str.split(expand=True).stack().value_counts().\
reset_index(name='freq').rename(columns={'index':'loanuse_term'}).shape#.\
#to_excel(my_folder+'2018_loanuse_lm2_wordfreq.xlsx')
df2018['LOAN_USE_LM3'].str.split(expand=True).stack().value_counts().\
reset_index(name='freq').rename(columns={'index':'loanuse_term'}).shape
df2018['LOAN_USE_LM3_V2'].str.split(expand=True).stack().value_counts().\
reset_index(name='freq').rename(columns={'index':'loanuse_term'}).shape

(6938, 2)

(2394, 2)

(2739, 2)

In [0]:
whos

In [0]:
#df2017.to_csv(my_folder+'loans_2017_v3.csv', index=False)
#df2018=df2018.drop(columns=[['Unnamed: 0','words']],axis=1)
df2018.columns
df2018.to_csv(my_folder+'loans_2018_v3.csv', index=False)

Index(['LOAN_ID', 'LOAN_NAME', 'ORIGINAL_LANGUAGE', 'DESCRIPTION',
       'DESCRIPTION_TRANSLATED', 'FUNDED_AMOUNT', 'LOAN_AMOUNT', 'STATUS',
       'IMAGE_ID', 'VIDEO_ID', 'ACTIVITY_NAME', 'SECTOR_NAME', 'LOAN_USE',
       'COUNTRY_CODE', 'COUNTRY_NAME', 'TOWN_NAME', 'CURRENCY_POLICY',
       'CURRENCY_EXCHANGE_COVERAGE_RATE', 'CURRENCY', 'PARTNER_ID',
       'POSTED_TIME', 'PLANNED_EXPIRATION_TIME', 'DISBURSE_TIME',
       'RAISED_TIME', 'LENDER_TERM', 'NUM_LENDERS_TOTAL',
       'NUM_JOURNAL_ENTRIES', 'NUM_BULK_ENTRIES', 'TAGS', 'BORROWER_NAMES',
       'BORROWER_GENDERS', 'BORROWER_PICTURED', 'REPAYMENT_INTERVAL',
       'DISTRIBUTION_MODEL', 'DESC_TRANSLATED_WORDCOUNT', 'DESC_WORDCOUNT',
       'DESCRIPTION_NEW', 'DESC_NEW_WORDCOUNT', 'LOAN_USE_WORDCOUNT',
       'AVY_NAME_WORDCOUNT', 'N_DESC_TRANSLATED', 'N_DESC', 'N_DESC_NEW',
       'N_BORROWERS', 'N_BORROWERS2', 'GROUP_LOANS', 'N_F_BORROWERS',
       'N_M_BORROWERS', 'HAVE_PICTURE', 'HAVE_VIDEO', 'N_BORROWERS_PICTURED',
      

### test whether can generate the crosstab or not 

In [0]:
df2017['words'] = df2017.LOAN_USE_LM3.str.strip().str.split('[\W_]+')
df2017.head()

In [0]:
#import datetime
start_time=datetime.datetime.now()

rows = list()
for row in df2017[['LOAN_ID', 'words']].iterrows():
    r = row[1]
    for word in r.words:
        rows.append((r.LOAN_ID, word))

words = pd.DataFrame(rows, columns=['LOAN_ID', 'words'])
print("time taken:",datetime.datetime.now()-start_time)
words['qty']=1
words.shape
words.head()

time taken: 0:01:01.584605


(1514745, 3)

,LOAN_ID,words,qty
0,1401164,purchase,1
1,1401164,pig,1
2,1401164,stock,1
3,1401164,vegetable,1
4,1401164,sale,1


In [0]:
basket_sets=pd.crosstab(words['LOAN_ID'],words['words'])
#basket_sets=pd.crosstab(words['words'],words['LOAN_ID'])
basket_sets.shape
basket_sets.head(5)

(225476, 2699)

words,,a,abaca,ability,able,abroad,abundant,academic,access,accessory,accommodate,account,accounting,ace,acetone,achieve,acquire,acquisition,acre,acrylic,activity,adapt,adapter,adaptor,add,addition,additional,adequate,adhesive,adjustment,administration,adult,advance,advantage,advertise,advertisement,advertising,afford,affordable,africa,...,wood,wooden,wool,woolen,work,worker,workforce,working,workingroom,workplace,workroom,workshop,world,would,woven,wrap,wraparound,wrapper,wrench,yam,yard,yarn,yea,year,yearend,yearâ,yeast,yellow,yield,yogurt,young,youth,yuca,yucca,zinc,zip,zipper,zoona,zoonabranded,â
LOAN_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1213974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1213975,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1213976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1213977,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1213978,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
for col in basket_sets.columns:
  if basket_sets[col].max()>1:
    print(col,":",basket_sets[col].max())

In [0]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

In [0]:
basket_sets = basket_sets.applymap(encode_units)

In [0]:
basket_sets.head(5)

words,,a,abaca,ability,able,abroad,abundant,academic,access,accessory,accommodate,account,accounting,ace,acetone,achieve,acquire,acquisition,acre,acrylic,activity,adapt,adapter,adaptor,add,addition,additional,adequate,adhesive,adjustment,administration,adult,advance,advantage,advertise,advertisement,advertising,afford,affordable,africa,...,wood,wooden,wool,woolen,work,worker,workforce,working,workingroom,workplace,workroom,workshop,world,would,woven,wrap,wraparound,wrapper,wrench,yam,yard,yarn,yea,year,yearend,yearâ,yeast,yellow,yield,yogurt,young,youth,yuca,yucca,zinc,zip,zipper,zoona,zoonabranded,â
LOAN_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1213974,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1213975,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1213976,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1213977,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1213978,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
#df2017=df2017.drop(columns=['words'],axis=1)
#df2017.columns
df2017.to_csv(my_folder+'loans_2017_v2.csv', index=False)

In [0]:
#https://www.w3resource.com/python-exercises/string/python-data-type-string-exercise-12.php

def word_count(str):
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return count
  
word_count('the quick brown fox jumps over the lazy dog.')

In [0]:
#https://stackoverflow.com/questions/46786211/counting-the-frequency-of-words-in-a-pandas-data-frame
#df.ACTIVITY_NAME.str.split(expand=True).stack().value_counts().to_excel(my_folder+'avy_name_wordfreq.xls')
df.LOAN_USE.str.split(expand=True).stack().value_counts().to_excel(my_folder+'loan_use_wordfreq.xlsx')
df.DESCRIPTION.str.split(expand=True).stack().value_counts().to_excel(my_folder+'desc_wordfreq.xlsx')
df.DESCRIPTION_NEW.str.split(expand=True).stack().value_counts().to_excel(my_folder+'desc_new_wordfreq.xlsx')
df.DESCRIPTION_TRANSLATED.str.split(expand=True).stack().value_counts().to_excel(my_folder+'desc_trans_wordfreq.xlsx')

## run the code below will need very high RAM

In [0]:
#from IPython.display import display, HTML
#RAM consuming so cannot use this, try explore an option from nltk, or remove stopwords first
df.DESCRIPTION_TRANSLATED.str.split(expand=True).stack().value_counts().to_frame().reset_index()#.\
#to_csv('/content/drive/My Drive/kiva data/check description field from loans file.csv')
#HTML(df3Desc.to_html())

In [0]:
#MAY WANT TO CONSIDER THE FOLLOWING TOO:borrower_pictured,borrower_genders, town_name
df[['STATUS','ACTIVITY_NAME','SECTOR_NAME','COUNTRY_NAME']]=\
df[['STATUS','ACTIVITY_NAME','SECTOR_NAME','COUNTRY_NAME']].astype('category')

### Text preprocessing using dask

https://extrapolations.dev/blog/2015/07/reproduceit-reddit-word-count-dask/

In [0]:
import re
import time
import nltk
import dask
import dask.bag as db
import nltk
from nltk.corpus import stopwords

In [0]:
nltk.download('stopwords')
nltk.download('punkt')  #to use word_tokenize
nltk.download('wordnet')  #to use WordNetLemmatizer

In [0]:
no_stopwords = lambda x: x not in stopwords.words('english')
is_word = lambda x: re.search("^[0-9a-zA-Z]+$", x) is not None